# Overview

Main topics of this notebooks are:

- Select different embedding models
- Use embeddings with Ollama for context awear invoke

## Build llm instance

In [1]:
from langchain.llms import Ollama
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

model_name = "mistral"

llm  = Ollama(
    model=model_name,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [2]:
llm.invoke("What is Ollama?")

 I'm unable to provide an answer for that specific question as "Ollama" doesn't appear to be a widely recognized term or concept in various fields such as technology, mathematics, science, or culture. It could potentially be a local or specialized term, but without more context it's difficult to determine what Ollama might refer to exactly. If you can provide some additional information or context about where or in what context this term is being used, I'd be happy to try and help out further!

' I\'m unable to provide an answer for that specific question as "Ollama" doesn\'t appear to be a widely recognized term or concept in various fields such as technology, mathematics, science, or culture. It could potentially be a local or specialized term, but without more context it\'s difficult to determine what Ollama might refer to exactly. If you can provide some additional information or context about where or in what context this term is being used, I\'d be happy to try and help out further!'

### Embeddings

In [3]:
from langchain_community.embeddings import OllamaEmbeddings

ollama_emb = OllamaEmbeddings(
    model=model_name,
)

In [4]:
r1 = ollama_emb.embed_documents(
    [
        "Alpha is the first letter of Greek alphabet",
        "Beta is the second letter of Greek alphabet",
    ]
)

In [5]:
r1

[[3.6778485774993896,
  -11.58975887298584,
  -1.3853873014450073,
  -9.210597038269043,
  -4.039162635803223,
  -10.6876220703125,
  0.1186661347746849,
  6.876765251159668,
  3.365912675857544,
  0.38536885380744934,
  -0.8180171847343445,
  1.4554296731948853,
  4.204746246337891,
  0.754840612411499,
  0.17539411783218384,
  0.34573784470558167,
  1.173801302909851,
  1.9944130182266235,
  2.6575396060943604,
  -9.223782539367676,
  1.4064568281173706,
  -7.66896915435791,
  11.173661231994629,
  0.5564149022102356,
  -6.734057426452637,
  -2.3574280738830566,
  -2.755812644958496,
  -2.93587589263916,
  4.756707191467285,
  -1.3174729347229004,
  9.799580574035645,
  -1.3128516674041748,
  5.251631259918213,
  3.829610824584961,
  -3.115138053894043,
  -0.9637590646743774,
  8.722522735595703,
  7.234260559082031,
  -0.612128496170044,
  7.075716018676758,
  3.0005722045898438,
  1.5532207489013672,
  7.538640022277832,
  -1.27712082862854,
  7.523481369018555,
  -1.72350263595581

In [6]:
r2 = ollama_emb.embed_query(
    "What is the second letter of Greek alphabet"
)

In [7]:
r2

[6.0437726974487305,
 -2.811547040939331,
 -1.713320016860962,
 -5.138332366943359,
 -3.7078261375427246,
 -9.134941101074219,
 4.115787029266357,
 2.310345411300659,
 -4.057328224182129,
 3.5155189037323,
 6.759912490844727,
 -5.8350725173950195,
 5.581575870513916,
 -0.8515759110450745,
 -6.9293389320373535,
 3.5821259021759033,
 -1.4296549558639526,
 2.321613311767578,
 1.5397531986236572,
 -6.815343379974365,
 3.939633846282959,
 4.051973819732666,
 -1.5510365962982178,
 -8.546457290649414,
 -7.411561489105225,
 -2.543473720550537,
 -0.17458251118659973,
 -2.4406492710113525,
 -1.1099518537521362,
 -3.499519109725952,
 10.73252010345459,
 -4.671807289123535,
 7.658094882965088,
 10.252423286437988,
 3.4262919425964355,
 -4.307258605957031,
 -0.2451966553926468,
 3.8035361766815186,
 -2.5096657276153564,
 5.711313247680664,
 7.770141124725342,
 -4.287423133850098,
 5.756283283233643,
 -0.6498416662216187,
 -4.138209342956543,
 -5.9587483406066895,
 -3.1635773181915283,
 0.1750063598

In [8]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/mantid-fork.pdf")
pages = loader.load_and_split()

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(pages)

In [9]:
from langchain.vectorstores import Chroma

target_source_chunks = 4

db = Chroma.from_documents(all_splits, ollama_emb)
retriever = db.as_retriever(search_kwargs={"k": target_source_chunks})

In [10]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
)

In [11]:
qa.invoke("What is ornl-next?")

 Ornl-next is a secondary branch in a GitHub repository used for development and testing of changes that have already been merged into the main branch. It's recommended for ORNL staff to create and manage this branch, and the workflow involves creating two PRs (one targeting main and one targetting ornl-next) and merging the PR into main first before merging it into ornl-next. The second PR can typically be auto-merged after checking that all requirements have been met.

{'query': 'What is ornl-next?',
 'result': " Ornl-next is a secondary branch in a GitHub repository used for development and testing of changes that have already been merged into the main branch. It's recommended for ORNL staff to create and manage this branch, and the workflow involves creating two PRs (one targeting main and one targetting ornl-next) and merging the PR into main first before merging it into ornl-next. The second PR can typically be auto-merged after checking that all requirements have been met."}

In [12]:
llm.invoke("What is ornl-next?")

 ORNL-Next is a computational tool developed by Oak Ridge National Laboratory (ORNL), which is a U.S. Department of Energy science and technology laboratory located in Oak Ridge, Tennessee. ORNL-Next is specifically designed for accelerating materials discovery using machine learning and high-performance computing. It provides a platform to screen and predict the properties of new materials based on their structural information, thereby enabling faster and more efficient material design and discovery process. This tool is particularly useful for researchers in materials science, chemistry, and related fields who seek to discover novel materials with specific desired properties.

' ORNL-Next is a computational tool developed by Oak Ridge National Laboratory (ORNL), which is a U.S. Department of Energy science and technology laboratory located in Oak Ridge, Tennessee. ORNL-Next is specifically designed for accelerating materials discovery using machine learning and high-performance computing. It provides a platform to screen and predict the properties of new materials based on their structural information, thereby enabling faster and more efficient material design and discovery process. This tool is particularly useful for researchers in materials science, chemistry, and related fields who seek to discover novel materials with specific desired properties.'